# P300 Collection as described by Krigolson et al.

In this notebook you can find a P300 acquisition protocoll based on the one described by Kringolson et al. in their paper "Choosing MUSE: Validation of a Low-Cost, Portable EEG System for ERP Research"

## Code for Experiment

### Imports

In [ ]:
import sys
sys.path.append("../")

# Imports
from neuropack.tasks.color_task import ProbabilisticColorTask
from neuropack.containers import EEGContainer
from neuropack.preprocessing.filters import BaselineCorrectionFilter, BandpassFilter, NotchFilter, ReductionFilter
from neuropack.preprocessing import PreprocessingPipeline
from neuropack.utils import osum
from neuropack.devices import BrainFlowDevice

from time import time
import os

### Creating data class, device, and filtering pipeline

In [ ]:
# Setup everything
muse_device = BrainFlowDevice.CreateMuse2Device()
eeg_container = EEGContainer(muse_device.channel_names, muse_device.sample_rate)

pipeline = PreprocessingPipeline()
pipeline.add_filter(NotchFilter())
pipeline.add_filter(BandpassFilter())
pipeline.add_filter(BaselineCorrectionFilter())
pipeline.add_filter(ReductionFilter("TP9", "TP10"))

In [ ]:
# connect the Muse
muse_device.connect()

### Setup task as based on task described by Krigolson et al.

In [ ]:
target_color = "blue"
non_target_color = "green"
task = ProbabilisticColorTask(0.25, non_target_color, target_color, (800, 1200), inter_stim_time=(300, 500))

### Experiment

In [ ]:
# Parameters
exp_duration_s = 30
output_filename = None

In [ ]:
# Setup datastructures needed for recording
eeg_container = EEGContainer(muse_device.channel_names, muse_device.sample_rate)
start = time()
times = []


# Start everything
muse_device.start_stream()
task.start()

# Main loop to collect device data. Do nothing else here to ensure all data is gathered
# and no data is lost to ringbuffer.
while time() - start < exp_duration_s:
    if muse_device.has_data():
        eeg_container.add_data(muse_device.fetch_data())
stop_time = time()
muse_device.stop_stream()


# After we have collected sufficient data, also gather data from event to figure out
# when stimuli were presented.
task_times = []
while task.has_data():
    t = task.fetch_data()
    if t.timestamp > stop_time:
        break
    task_times.append(t)
task.stop()

In [ ]:
# Only get tasks which where stimuli
task_times = [x.timestamp for x in task_times if x.is_target]

# Add events to the EEG container
for x in task_times:
   eeg_container.mark_event(1, x) 

task_events = eeg_container.get_events(1, 100, 800)

# Remove events which are to close to the start or end of the recording
while len(task_events[-1]) != len(task_events[0]):
    task_events.pop()

### Preprocessing and plotting

In [ ]:
pipeline.apply(task_events)
avg_event = osum(task_events) / len(task_events)

avg_event.plot_ch()

[2023-05-25 23:58:55.956] [board_logger] [error] failed to unsubscribe for 0000fe8d-0000-1000-8000-00805f9b34fb 273e0009-4c4d-454d-96be-f03bac821358
[2023-05-25 23:58:55.957] [board_logger] [error] failed to unsubscribe for 0000fe8d-0000-1000-8000-00805f9b34fb 273e000a-4c4d-454d-96be-f03bac821358
[2023-05-25 23:58:55.958] [board_logger] [error] failed to unsubscribe for 0000fe8d-0000-1000-8000-00805f9b34fb 273e0003-4c4d-454d-96be-f03bac821358
[2023-05-25 23:58:55.960] [board_logger] [error] failed to unsubscribe for 0000fe8d-0000-1000-8000-00805f9b34fb 273e0004-4c4d-454d-96be-f03bac821358
[2023-05-25 23:58:55.961] [board_logger] [error] failed to unsubscribe for 0000fe8d-0000-1000-8000-00805f9b34fb 273e0005-4c4d-454d-96be-f03bac821358
[2023-05-25 23:58:55.962] [board_logger] [error] failed to unsubscribe for 0000fe8d-0000-1000-8000-00805f9b34fb 273e0006-4c4d-454d-96be-f03bac821358
[2023-05-25 23:58:55.963] [board_logger] [error] failed to unsubscribe for 0000fe8d-0000-1000-8000-00805f9

### Saving EEG recording

In [ ]:
file_name = "my_data.csv"
eeg_container.save_signals(os.path.join("rec", file_name))

## References

[1] O. E. Krigolson, C. C. Williams, A. Norton, C. D. Hassall, and F. L. Colino, “Choosing MUSE: Validation of a Low-Cost, Portable EEG System for ERP Research,” Front. Neurosci., vol. 11, Mar. 2017, doi: 10.3389/fnins.2017.00109.
